In [2]:
import numpy as np
import copy
import pandas as pd
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

from scipy.stats import invwishart, invgamma,multivariate_normal
from utils import DagMake
from mcem import mcem

In [3]:
# reparametrizing sigma_h to unconstrained parameters
sigma_h_to_V= tfb.Chain([  
    tfb.TransformDiagonal(tfb.Invert(tfb.Exp())),
    tfb.Invert(tfb.CholeskyOuterProduct()),
])
flatten = tfb.Chain([
    tfb.Invert(tfb.FillTriangular()),    
])

def shrinkage(sigma,p,s):
    omega=np.linalg.inv(sigma)
    for i in range(0,p):
        for j in range (0,p):
                    if omega[i,j]<=s:
                        omega[i,j]=0
    return omega

def structure(omega,p):
    G=omega
    for i in range(0,p):
        for j in range (0,p):
            if i==j:
                G[i,j]=0
            else:
                if G[i,j]!=0:
                    G[i,j]=1               
    return G

In [26]:
# Initial lower-traguular matrix(which is the unrestricted variables to \sigma_h)
V= np.eye(10, dtype=np.float32)

# Transfer unrestricted matrix V to \sigma_h (positive definited matrix)
sigma_h= sigma_h_to_V.inverse(V).numpy()

# Generated $m$ covriance matrices from \sigma_h 
# Here we generate 3 covariance matrices form an inverse Wishart distribution
#      with degree of freedom equal to 15, and covariance equal to sigma_h
sigma_l=invwishart.rvs(15, sigma_h, size=3, random_state=179)

In [27]:
# Shrink some elements of the precision matrix with small absolute value to zero
omega_pre=copy.deepcopy(sigma_l)
for i in range (0,3):
    omega_pre[i]=shrinkage(omega_pre[i],10,0.4)
    
# Generate the corresponding graph structure from omega_pre
omega_structure=copy.deepcopy(omega_pre)
for i in range (0,3):
    omega_structure[i]=structure(omega_structure[i],10)
    
# Generate the corresponding covariance matrix from the sparse precision matrix 
sigma_sparse=copy.deepcopy(omega_pre)
for i in range (0,3):
    sigma_sparse[i]=np.linalg.inv(omega_pre[i])

In [31]:
# Initialization

m=3 #the number of tasks 
v0=15 #degree of freedom in the inverse Wishart distribution 
p=10 #dimension
stepsize=0.0000021 #the step length in HMC algorithm 
iteration=5 #the number of iterations in HMC algorithm
epsilon=0.0005
# Generate datasets
dataset=dict()
initial_G=dict()
for i in range(1,m+1):
    dataset[i]=np.array(multivariate_normal.rvs(mean=None, cov=sigma_sparse[i-1], size=250, random_state=134),'float32')
    initial_G[i] =np.array(DagMake(10),'float32')

In [32]:
BN=mcem(m,v0,p,V,stepsize,iteration,dataset,initial_G,epsilon)

In [40]:
BN

{1: array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32),
 2: array([[0., 1., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.